In [1]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal
from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [7]:
#schema 

class SentimentSchema(BaseModel):

    sentiment: Literal['positive','negative'] = Field(description="Sentiment of the review")

In [ ]:
#diagnosis schema

class DiagnosisSchema(BaseModel):
    issue_type: Literal['UX','Performance','Bug','Support','Others'] = Field(description="The category of the issue mentioned in the review.")

    tone: Literal['angry','frustated','disappointed','normal'] = Field(description="The emotional tone expressed by the user")
    
    urgency: Literal['low','medium','high'] = Field(description="How urgent or critical the issue appears to be") 

In [15]:
structured_llm1 = llm.with_structured_output(SentimentSchema)
structured_llm2 = llm.with_structured_output(DiagnosisSchema)

In [10]:
prompt = "The software is too bad!"

structured_llm.invoke(prompt).sentiment

'negative'

In [11]:
class ReviewState(TypedDict):

    review: str
    sentiment: Literal['positive', 'negative']

    diagnosis: dict

    response: str


In [12]:
def find_sentiment(state: ReviewState) ->ReviewState:

    prompt = f"For the following review find out the sentiment.\n{state['review']}"

    sentiment = structured_llm.invoke(prompt).sentiment

    return {'sentiment': sentiment}

In [ ]:
def check_sentiment(state: ReviewState) -> Literal["positive_response", "run_diagnosis"]:

    if state['sentiment'] == "positive":
        return "positive_response"
    else:
        return "run_diagnosis"

In [ ]:
def positive_resposne(state: ReviewState) ->ReviewState:

    promt = f"Write a warm thank you message in response to this review: \n\n{state['review']}\nAlso kindly ask the user to leave feedback on our website."

    response = llm.invoke(prompt).content

    return {'response': response}

In [ ]:
def run_diagnosis(state: ReviewState) ->ReviewState:

    prompt = f"Diagnose this negative review:\n\n{state['review']}\nRetrun issue_type, tone, and urgency"

    response = structured_llm2.invoke(prompt)

    return {'diagnosis': response.llm.dump()}

def negative_response(state: ReviewState) ->ReviewState:

    prompt = f''' You are a support assistant. The user had a {diagnosis['issue_type']} issue, sounded'{diagnosis['tone']}'''

In [ ]:
graph  = StateGraph(ReviewState)

#node
graph.add_node('find_sentiment', find_sentiment)

graph.add_node('positive_response', positive_response)
graph.add_node('run_diagnosis', run_diagnosis)
graph.add_node('negative_response', negative_response)


#edges
graph.add_edge(START, 'find_sentiment')